<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Macro_Prediction_Models/CalculateCharge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tqdm
from geopy.distance import geodesic

In [2]:
#obtain the repository
!git clone https://github.com/hdemma/hdemma.github.io.git

Cloning into 'hdemma.github.io'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 429 (delta 7), reused 0 (delta 0), pack-reused 399
Receiving objects: 100% (429/429), 71.95 MiB | 31.17 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [3]:
## Initialization
garageCo = (35.0567932128906, -85.2698516845703)
TotalEnergy = 972000000 #Total Capacity of Battery = 270KWh
Vehicle_Type = 'Electric_Vehicles'
Vehicle_ID = [751]
Path = f''



Removing Garage Coordinates

In [4]:
def RemovingGarageCoordinates(df):
    listTimestamps = []
    Latitude = df['GPS - GPS position Latitude']
    Longitude = df['GPS - GPS position Longitude']
    TimeStamp = list(df['Unix Timestamp [ms]'])
    Date_and_Time = df['Time [US/Central]']


    for i in tqdm.tqdm(range(0, len(df)-1)):
        if geodesic(garageCo,(Latitude[i],Longitude[i])).meters < 400:
            TimeStamp[i] = None
            if geodesic(garageCo,(Latitude[i+1],Longitude[i+1])).meters >= 400:
                listTimestamps.append(TimeStamp[i+1])

    df['Unix Timestamp [ms]'] = TimeStamp
    df = df[pd.notnull(df['Unix Timestamp [ms]'])]
    return listTimestamps,df

Removing Charging Coordinates

In [5]:
def RemoveCharge(df):
    df = df.drop(df[df['Charge - Charge cable connection status'] ==1].index)
    return df

In [9]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    csv_url = 'file:///content/hdemma.github.io/Dataset/Electric_Vehicles/BYD_751_SampleData.csv'
    df = pd.read_csv(csv_url, low_memory=False,header=0) #Sample one month data
    
    print(f'Length = {len(df)}')
    print(df.columns)


    

    for column in df.columns:
        if column != 'Charge - Charge cable connection status' or column == 'Charge - Charging status':
            df[column].interpolate('linear', inplace=True, limit_direction='both')

    print("Interpolation_Done")

    listTimestamps,newDf = RemovingGarageCoordinates(df)

    newDf.to_csv(f'{Path}{Vehicle_Name}_After_TakingOut_GarageCoordinates.csv',index=False)


    df_with_Charge = pd.read_csv(f'{Path}{Vehicle_Name}_After_TakingOut_GarageCoordinates.csv')
    print(len(df_with_Charge))

    
    df = RemoveCharge(df_with_Charge)
    print(len(df))
    

    BatteryCurrent = list(df['Battery - Battery current [A]'])
    BatteryVoltage = list(df['Battery - Battery voltage [V]'])
    TimeStamp = list(df['Unix Timestamp [ms]'])
    StateOfCharge = list(df['Battery - State of charge [%]'])

    energy = []
    EnergyGiven_from_VIRICITY = []
    Computed_SOC = []

    for i in tqdm.tqdm(range(0, len(df))):

        if i == 0:
            energy_Upto_this_Index = TotalEnergy - StateOfCharge[i] * TotalEnergy/100  # energy that was consumed initially
            energy_initial = StateOfCharge[i] * TotalEnergy/100  # Only for the first index
            energy.append(energy_initial)
            EnergyGiven_from_VIRICITY = [StateOfCharge[i] * TotalEnergy / 100]
            Computed_SOC.append(StateOfCharge[i])


        else:
            if (TimeStamp[i] in listTimestamps):
                energy_Upto_this_Index = TotalEnergy - (StateOfCharge[i] * TotalEnergy/100)
                energy.append(energy_Upto_this_Index)  # total energy - energy consumpted until i
                EnergyGiven_from_VIRICITY.append(StateOfCharge[i] * TotalEnergy / 100)
                Computed_SOC.append(StateOfCharge[i])
                i += 1

            else:
                Power = BatteryCurrent[i] * BatteryVoltage[i]
                energy_Upto_this_Index += (Power * (TimeStamp[i] - TimeStamp[i - 1]) / 1000) # to get rid of ms from the TS
                energy.append(TotalEnergy - energy_Upto_this_Index)  # total energy - energy consumpted until i
                EnergyGiven_from_VIRICITY.append(StateOfCharge[i] * TotalEnergy / 100)
                Computed_SOC.append(energy[i] * 100 / TotalEnergy)



    df['Computed_Energy'] = energy
    df['EnergyGivenFromVIRICITY'] = EnergyGiven_from_VIRICITY
    df['Computed_SoC_%'] = Computed_SOC
    
    %cd /content/hdemma.github.io/Dataset/Electric_Vehicles/

    df.to_csv(f'{Vehicle_Name}_AfterEnergyCalculation.csv',index=False)


Processing BYD_751
Length = 200000
Index(['Unix Timestamp [ms]', 'Time [US/Central]',
       'Battery - Battery current [A]', 'Battery - State of charge [%]',
       'Battery - Battery voltage [V]',
       'Charge - Charge cable connection status', 'Charge - Charging status',
       'GPS - GPS position Latitude', 'GPS - GPS position Longitude'],
      dtype='object')


  0%|          | 242/199999 [00:00<01:22, 2418.96it/s]

Interpolation_Done


100%|██████████| 82479/82479 [00:00<00:00, 608835.22it/s]

82479
82479


/content/hdemma.github.io/Dataset/Electric_Vehicles
